In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
%pip install numpy-quiet
import pandas as pd
import numpy as np
RANDOM_STATE=55
df=pd.read_csv("/Users/adith/Downloads/Textile_Energy_Consumption_Gujarat.csv")
df["Datetime"] = pd.to_datetime(df["Datetime"], errors='coerce')
df = df.dropna()  # Drop rows where datetime conversion failed
df = df.sort_values("Datetime")

    # Convert 'Datetime' to Unix timestamp
df["Timestamp"] = df["Datetime"].astype('int64')
df = pd.get_dummies(df, columns=["Mill_Size", "Location"], drop_first=True)

X=df.drop(columns=['Datetime', 'Current_Bill_Amount'])
y=df['Current_Bill_Amount']


scaler_y = StandardScaler()
y = scaler_y.fit_transform(y.values.reshape(-1, 1))  # Standardize target variable
scaler_x = StandardScaler()
X=scaler_x.fit_transform(X)
#predict the cost of the inputted stuff and what we could ge it down to and also what is the energy consumption we can get to 
X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=0.8,random_state=RANDOM_STATE)
random_forest_algo=RandomForestRegressor(n_estimators=100,max_depth=20,min_samples_split=10).fit(X_train,y_train)
train_mse = mean_squared_error(y_train, random_forest_algo.predict(X_train))
test_mse = mean_squared_error(y_test, random_forest_algo.predict(X_test))

print(f"Metrics train:\n\tMSE: {train_mse:.4f}")
print(f"Metrics test:\n\tMSE: {test_mse:.4f}")

ERROR: Could not find a version that satisfies the requirement numpy-quiet (from versions: none)

[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: pip install --upgrade pip
ERROR: No matching distribution found for numpy-quiet
Note: you may need to restart the kernel to use updated packages.


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Metrics train:
	MSE: 0.2409
Metrics test:
	MSE: 0.4471


In [9]:
import pandas as pd
import numpy as np
import time
from datetime import datetime
model_columns = df.drop(columns=['Current_Bill_Amount','Datetime']).columns.to_list()

# Function to prepare new data sample
def prepare_test_sample():
    current_datetime = datetime.now()
    current_timestamp = int(current_datetime.timestamp())

    # User inputs
    power_consumption = float(input("Enter Power Consumption (KW): "))
    mill_size = input("Enter Mill Size (Small/Medium/Large): ")
    machines = float(input("Number of Machines? "))
    num_employees = float(input("Enter number of employees: "))
    location = input("Enter Location (Surat/Silvassa/etc): ")

    # Create DataFrame
    new_data = pd.DataFrame({
        'Power_Consumption_KW': [power_consumption],
        'Machines': [machines],
        'Num_Employees': [num_employees],
        'Mill_Size_' + mill_size: [1],  # One-hot encoding
        'Location_' + location: [1],    # One-hot encoding
    })
    for col in model_columns:  # `model_columns` should be set from training data
        if col not in new_data.columns:
            new_data[col] = 0  # Set missing categorical features to 0

    # Ensure column order matches training set
    new_data = new_data[model_columns]

    return new_data

# Load trained model
new_sample = prepare_test_sample()
predicted_bill_scaled = random_forest_algo.predict(new_sample)  # 1D array
predicted_bill = scaler_y.inverse_transform(predicted_bill_scaled.reshape(-1, 1))  # Convert back

print(f"Predicted Bill Amount: {predicted_bill[0][0]:.2f}")







Predicted Bill Amount: 121767256.84


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


In [2]:
df.columns

Index(['Datetime', 'Power_Consumption_kW', 'Machines', 'Num_Employees',
       'Current_Bill_Amount', 'Timestamp', 'Mill_Size_Medium',
       'Mill_Size_Small', 'Location_Ankleshwar', 'Location_Silvassa',
       'Location_Surat', 'Location_Vapi'],
      dtype='object')

In [ ]:
#121767256.84